In [1]:
     !pip install mindspore-2.2.13-cp39-cp39-linux_aarch64.whl -i https://pypi.tuna.tsinghua.edu.cn/simple

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Processing ./mindspore-2.2.13-cp39-cp39-linux_aarch64.whl
  Attempting uninstall: mindspore
    Found existing installation: mindspore 2.2.0
    Uninstalling mindspore-2.2.0:
      Successfully uninstalled mindspore-2.2.0
You should consider upgrading via the '/home/ma-user/anaconda3/envs/MindSpore/bin/python3.9 -m pip install --upgrade pip' command.


In [2]:
!pip install pytest==7.2.*

Looking in indexes: http://pip.modelarts.private.com:8888/repository/pypi/simple
     |████████████████████████████████| 317 kB 40.9 MB/s eta 0:00:01
  Attempting uninstall: pytest
    Found existing installation: pytest 7.1.2
    Uninstalling pytest-7.1.2:
      Successfully uninstalled pytest-7.1.2
You should consider upgrading via the '/home/ma-user/anaconda3/envs/MindSpore/bin/python3.9 -m pip install --upgrade pip' command.


In [3]:
!pip install mindnlp

Looking in indexes: http://pip.modelarts.private.com:8888/repository/pypi/simple
     |████████████████████████████████| 5.7 MB 32.0 MB/s eta 0:00:01
     |████████████████████████████████| 547 kB 58.1 MB/s eta 0:00:01
     |████████████████████████████████| 316 kB 58.7 MB/s eta 0:00:01
     |████████████████████████████████| 2.2 MB 52.7 MB/s eta 0:00:01
     |████████████████████████████████| 3.6 MB 33.0 MB/s eta 0:00:01
     |████████████████████████████████| 1.2 MB 57.1 MB/s eta 0:00:01
     |████████████████████████████████| 84 kB 38.4 MB/s eta 0:00:01
     |████████████████████████████████| 1.2 MB 37.7 MB/s eta 0:00:01
     |████████████████████████████████| 38.1 MB 56.9 MB/s eta 0:00:01
     |████████████████████████████████| 78 kB 59.3 MB/s eta 0:00:01
     |████████████████████████████████| 402 kB 64.0 MB/s eta 0:00:01
     |████████████████████████████████| 64 kB 52.7 MB/s eta 0:00:01
     |████████████████████████████████| 220 kB 57.6 MB/s eta 0:00:01
     |██████████████████

In [10]:
import os

import mindspore
from mindspore.dataset import text, GeneratorDataset, transforms
from mindspore import nn

from mindnlp.dataset import load_dataset

from mindnlp._legacy.engine import Trainer, Evaluator
from mindnlp._legacy.engine.callbacks import CheckpointCallback, BestModelCallback
from mindnlp._legacy.metrics import Accuracy

In [11]:
imdb_ds = load_dataset('imdb', split=['train', 'test'])
imdb_train = imdb_ds['train']
imdb_test = imdb_ds['test']

Generating unsupervised split: 100%|██████████| 50000/50000 [00:00<00:00, 144011.62 examples/s]


In [12]:
imdb_train.get_dataset_size()

25000

In [13]:
import numpy as np

def process_dataset(dataset, tokenizer, max_seq_len=512, batch_size=4, shuffle=False):
    is_ascend = mindspore.get_context('device_target') == 'Ascend'
    def tokenize(text):
        if is_ascend:
            tokenized = tokenizer(text, padding='max_length', truncation=True, max_length=max_seq_len)
        else:
            tokenized = tokenizer(text, truncation=True, max_length=max_seq_len)
        return tokenized['input_ids'], tokenized['attention_mask']

    if shuffle:
        dataset = dataset.shuffle(batch_size)

    # map dataset
    dataset = dataset.map(operations=[tokenize], input_columns="text", output_columns=['input_ids', 'attention_mask'])
    dataset = dataset.map(operations=transforms.TypeCast(mindspore.int32), input_columns="label", output_columns="labels")
    # batch dataset
    if is_ascend:
        dataset = dataset.batch(batch_size)
    else:
        dataset = dataset.padded_batch(batch_size, pad_info={'input_ids': (None, tokenizer.pad_token_id),
                                                             'attention_mask': (None, 0)})

    return dataset

In [14]:
from mindnlp.transformers import GPTTokenizer
# tokenizer
gpt_tokenizer = GPTTokenizer.from_pretrained('openai-gpt')

# add sepcial token: <PAD>
special_tokens_dict = {
    "bos_token": "<bos>",
    "eos_token": "<eos>",
    "pad_token": "<pad>",
}
num_added_toks = gpt_tokenizer.add_special_tokens(special_tokens_dict)

100%|██████████| 25.0/25.0 [00:00<00:00, 32.8kB/s]
797kB [00:00, 1.17MB/s]
448kB [00:00, 1.30MB/s]
1.21MB [00:01, 773kB/s]
656B [00:00, 1.54kB/s]                   
ftfy or spacy is not installed using BERT BasicTokenizer instead of SpaCy & ftfy.


In [16]:
# split train dataset into train and valid datasets
imdb_train, imdb_val = imdb_train.split([0.7, 0.3])

[WARNING] ME(23837:281473712357552,MainProcess):2024-07-07-18:55:26.729.867 [mindspore/dataset/engine/datasets.py:2480] Dataset is shuffled before split.


In [17]:
dataset_train = process_dataset(imdb_train, gpt_tokenizer, shuffle=True)
dataset_val = process_dataset(imdb_val, gpt_tokenizer)
dataset_test = process_dataset(imdb_test, gpt_tokenizer)

In [18]:
next(dataset_train.create_tuple_iterator())

[Tensor(shape=[16, 512], dtype=Int64, value=
 [[  616,  4121,   980 ... 40480, 40480, 40480],
  [  249,   976,  4336 ... 40480, 40480, 40480],
  [  507,   256,   252 ... 40480, 40480, 40480],
  ...
  [  616,   544, 37295 ... 40480, 40480, 40480],
  [  685,   240,   481 ... 40480, 40480, 40480],
  [  249,   509,   963 ... 40480, 40480, 40480]]),
 Tensor(shape=[16, 512], dtype=Int64, value=
 [[1, 1, 1 ... 0, 0, 0],
  [1, 1, 1 ... 0, 0, 0],
  [1, 1, 1 ... 0, 0, 0],
  ...
  [1, 1, 1 ... 0, 0, 0],
  [1, 1, 1 ... 0, 0, 0],
  [1, 1, 1 ... 0, 0, 0]]),
 Tensor(shape=[16], dtype=Int32, value= [0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0])]

In [19]:
from mindnlp.transformers import GPTForSequenceClassification
from mindspore.experimental.optim import Adam

# set bert config and define parameters for training
model = GPTForSequenceClassification.from_pretrained('openai-gpt', num_labels=2)
model.config.pad_token_id = gpt_tokenizer.pad_token_id
model.resize_token_embeddings(model.config.vocab_size + 3)

optimizer = nn.Adam(model.trainable_params(), learning_rate=2e-5)

metric = Accuracy()

# define callbacks to save checkpoints
ckpoint_cb = CheckpointCallback(save_path='checkpoint', ckpt_name='gpt_imdb_finetune', epochs=1, keep_checkpoint_max=2)
best_model_cb = BestModelCallback(save_path='checkpoint', ckpt_name='gpt_imdb_finetune_best', auto_load=True)

trainer = Trainer(network=model, train_dataset=dataset_train,
                  eval_dataset=dataset_train, metrics=metric,
                  epochs=3, optimizer=optimizer, callbacks=[ckpoint_cb, best_model_cb],
                  jit=False)

100%|██████████| 457M/457M [00:35<00:00, 13.4MB/s] 
100%|██████████| 74.0/74.0 [00:00<00:00, 95.1kB/s]
The following parameters in models are missing parameter:
['score.weight']


In [ ]:
trainer.run(tgt_columns="labels")

The train will start from the checkpoint saved in 'checkpoint'.


Epoch 0:   0%|          | 0/766 [00:00<?, ?it/s]

Epoch 0:  35%|███▌      | 271/766 [18:20<31:15,  3.79s/it, loss=0.34945548] 

In [14]:
evaluator = Evaluator(network=model, eval_dataset=dataset_test, metrics=metric)
evaluator.run(tgt_columns="labels")

Evaluate: 100%|█████████▉| 1562/1563 [06:28<00:00, 10.16it/s]

Evaluate: 100%|██████████| 1563/1563 [06:38<00:00,  3.92it/s]

Evaluate Score: {'Accuracy': 0.92768}
